# FoCS Project - Solution

You have to work on the [Kiva](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view) dataset. Some information on the datasets are on the [Kaggle](https://www.kaggle.com/) web page.

## Task 1

1. Normalize the `loan_lenders` table. In the normalized table, each row must have one `loan_id` and one lender.


The first step is to read the `loan_lenders.csv` file and create a dataframe with unnormalized data. Since the file is very large, I introduced a bollean variable `FINAL` - if it is false then only part of data is processed in order to spead up the work. Libraries `panda` and `numpy` will be used for data processing while the efficiency will be measured using the `time` library.

In [4]:
import pandas as pd
import numpy as np
import time as tm
FINAL = False # This is final calculation, on complete data
start_time = tm.time()
loans_lenders_un = pd.read_csv('./data/loans_lenders.csv')
if not FINAL:
    loans_lenders_un = loans_lenders_un.head(10000) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lenders_un

--- 14.587375402450562 seconds ---


,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
9995,45940,"helga4707, james6963, jimjams, andreas2382, si..."
9996,247491,"priyaram, christian9832, john9242, sandra1434,..."
9997,345274,"priyaram, nicola1093, bobby9744, simon7848, di..."
9998,125945,"joseph1859, matt5349, reese3555, stanley3312, ..."


In [ ]:
loans_lenders_normalized = 